In [51]:
import pandas as pd
import numpy as np
from openai import OpenAI, Embedding
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import os


In [52]:
df = pd.read_csv("../word-dropout/11_20_26_27_Articles_DROPOUT.csv") 
queries = pd.read_csv("../queries_for_documents.csv")
#openai_client = OpenAI(api_key = 'insert-api-key')

In [53]:
df.head()

,id,title,url,date,content,reduced_content_p=0.1,reduced_content_p=0.3,reduced_content_p=0.5,reduced_content_p=0.7,reduced_content_p=0.9
0,0,Yahoo Finance,https://finance.yahoo.com/news/threads-now-let...,"Wed, Nov 27, 2024, 12:15 PM",Threads is rolling out a redesign that keeps y...,Threads is a redesign keeps your feeds visible...,Threads rolling out a redesign that your visib...,"Threads is out a on home screen, head Adam ann...",is your the announced on The allows between ju...,redesign Instagram head you out can a should m...
1,1,Yahoo Finance,https://finance.yahoo.com/news/elon-musk-calls...,"Wed, Nov 27, 2024, 1:34 PM","Billionaire Elon Musk, a key adviser to the in...","Billionaire Musk, a key adviser to the incomin...","Elon Musk, a key adviser the incoming administ...","Elon adviser to the administration, called the...",adviser the incoming Trump the Protection to b...,Billionaire Trump for Consumer to setting on t...
2,2,Yahoo Finance,https://finance.yahoo.com/news/tpg-angelo-gord...,"Wed, Nov 27, 2024, 11:23 AM",(Bloomberg) -- Americans borrowing against the...,(Bloomberg) -- Americans borrowing against the...,(Bloomberg) borrowing against homes after a su...,their after surge in creating trillion opportu...,"borrowing their homes prices a investors, acco...",against a in prices Gordon. Brace Are Lanes eq...
3,3,Yahoo Finance,https://finance.yahoo.com/news/microsoft-faces...,"Wed, Nov 27, 2024, 12:58 PM",(Bloomberg) -- The US Federal Trade Commission...,(Bloomberg) -- The US Federal Trade Commission...,(Bloomberg) The Trade has an investigation of ...,(Bloomberg) -- Federal Commission has opened a...,The opened of from computing and licensing art...,and artificial Read Kansas Takes according to ...
4,4,Yahoo Finance,https://finance.yahoo.com/news/crowdstrike-fal...,"Wed, Nov 27, 2024, 6:40 AM",(Bloomberg) -- CrowdStrike Holdings Inc. fell ...,(Bloomberg) -- CrowdStrike Holdings Inc. fell ...,(Bloomberg) CrowdStrike Holdings in trading on...,(Bloomberg) -- Holdings Inc. in on after the c...,(Bloomberg) early the cybersecurity company we...,cybersecurity weaker-than-expected who compute...


In [54]:
queries.head()

,query,doc_index
0,Why is Australia's monetary policy still seen ...,551
1,What's Palo Alto Networks' revenue forecast fo...,400
2,How could institutional investments affect Bit...,377
3,"Hey, I've been hearing that some AI companies ...",240
4,What impact could increased tariffs have on th...,516


In [55]:
from openai import OpenAI, Embedding


def summarize_doc(openai_client, text):
    query = "Summarize this into less than 10 words"

    prompt = f"Context:\n{text}\n\nQuestion: {query}\nAnswer:"


    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
        {"role": "user", "content": prompt}
        ]
    )

    print(completion.choices[0].message)

In [56]:
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def preprocess_text(text):
    """Preprocess a single text input (either document or query) and use word stems."""
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')  # Add custom stop words if needed
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # Replace punctuation with spaces
    text_nopunct = text.translate(translator)  # Remove punctuation
    words_list = text_nopunct.strip().split()  # Tokenize into words
    stemmer = PorterStemmer()
    filtered_words = [stemmer.stem(word.lower()) for word in words_list if word.lower() not in stop_words and len(word) > 1]
    return filtered_words

def preprocess_df(df):
    """Preprocess a DataFrame column containing document content."""
    df["content"] = df["content"].apply(preprocess_text)
    for p in [0.1, 0.3, 0.5, 0.7, 0.9]:
        df[f"reduced_content_p={p}"] = df[f"reduced_content_p={p}"].apply(preprocess_text)
    return df

def preprocess_query(query):
    """Preprocess a query string."""
    return preprocess_text(query)


In [57]:
import bm25
import importlib

# Reload the module after making changes
importlib.reload(bm25)

# Initialize the BM25 instance

def rank_query_bm25(bm25, query, ground_truth_index, top_k=5, summarize_docs=True, only_scores=False):
    # Display initial document details

    if not only_scores:
        print("=" * 50)
        print("📄 Document Sample:")
        print(df.head())
        print("=" * 50)
        print(f"📏 Total Documents: {len(bm25_instance.doc_lengths)}")
        print(f"Preprocessed query: {query} with doc_index {ground_truth_index}")
    # Rank documents for a given query
    ranked_indices, scores = bm25_instance.rank(query)

    ranked_indices = ranked_indices[:top_k]
    scores = scores[:top_k]

    if not only_scores:
        print("\n🏆 Ranking Results:")
        print("=" * 50)
        for i, (doc_index, score) in enumerate(zip(ranked_indices, scores), start=1):
            print(f"⭐ Rank {i}:")
            print(f"   - Document Index: {doc_index}")
            print(f"   - BM25 Score: {score:.4f}")
            doc_content = df.iloc[doc_index]["content"]
            if (summarize_docs):
                summmary = summarize_doc(openai_client, doc_content)
                print(f"   - Document Content:\n{summmary}")
            else:
                print(f"   - Document Content:\n{doc_content}")
            print("-" * 50)
    return ranked_indices, scores

    
    


In [58]:
df = preprocess_df(df)
queries = queries[25:]

In [59]:
df.head()

,id,title,url,date,content,reduced_content_p=0.1,reduced_content_p=0.3,reduced_content_p=0.5,reduced_content_p=0.7,reduced_content_p=0.9
0,0,Yahoo Finance,https://finance.yahoo.com/news/threads-now-let...,"Wed, Nov 27, 2024, 12:15 PM","[thread, roll, redesign, keep, feed, visibl, h...","[thread, redesign, keep, feed, visibl, home, i...","[thread, roll, redesign, visibl, head, adam, m...","[thread, home, screen, head, adam, announc, re...","[announc, allow, roll, android, come, day, fol...","[redesign, instagram, head, much, test, option..."
1,1,Yahoo Finance,https://finance.yahoo.com/news/elon-musk-calls...,"Wed, Nov 27, 2024, 1:34 PM","[billionair, elon, musk, key, advis, incom, tr...","[billionair, musk, key, advis, incom, trump, a...","[elon, musk, key, advis, incom, administr, cal...","[elon, advis, administr, call, financi, protec...","[advis, incom, trump, protect, elimin, often, ...","[billionair, trump, consum, set, fellow, finan..."
2,2,Yahoo Finance,https://finance.yahoo.com/news/tpg-angelo-gord...,"Wed, Nov 27, 2024, 11:23 AM","[bloomberg, american, borrow, home, surg, real...","[bloomberg, american, borrow, surg, real, esta...","[bloomberg, borrow, home, surg, real, estat, p...","[surg, creat, trillion, opportun, accord, ange...","[borrow, home, price, investor, accord, tpg, r...","[price, gordon, brace, lane, equiti, tj, durki..."
3,3,Yahoo Finance,https://finance.yahoo.com/news/microsoft-faces...,"Wed, Nov 27, 2024, 12:58 PM","[bloomberg, us, feder, trade, commiss, open, a...","[bloomberg, us, feder, trade, commiss, open, a...","[bloomberg, trade, investig, microsoft, corp, ...","[bloomberg, feder, commiss, open, antitrust, m...","[open, comput, licens, artifici, read, york, c...","[artifici, read, kansa, take, accord, compani,..."
4,4,Yahoo Finance,https://finance.yahoo.com/news/crowdstrike-fal...,"Wed, Nov 27, 2024, 6:40 AM","[bloomberg, crowdstrik, hold, inc, fell, earli...","[bloomberg, crowdstrik, hold, inc, fell, earli...","[bloomberg, crowdstrik, hold, trade, wednesday...","[bloomberg, hold, inc, compani, issu, weaker, ...","[bloomberg, earli, cybersecur, compani, weaker...","[cybersecur, weaker, expect, comput, city’, op..."


In [65]:
import numpy as np
import time

# Assuming bm25_instance, preprocess_query, rank_query_bm25, and other dependencies are defined

# Initialize results and performance arrays
num_iterations = 50  # Number of iterations
results = np.zeros((6, 3))  # For storing Precision at 10, 5, 3, 1 for each p
average_times = np.zeros(6)  # To store average times over all iterations

# Mocking `queries` and `df` for demonstration purposes
p_values = [0, 0.1, 0.3, 0.5, 0.7, 0.9]

# Repeat the experiment 25 times
for iteration in range(num_iterations):
    i = 0
    current_docs = df["content"]

    for p in p_values:
        if p > 0:
            current_docs = df[f"reduced_content_p={p}"]  # Get the document content for this p-value

        bm25_instance = bm25.BM25(current_docs, k1=1.5, b=0.75)

        top1, top3, top5 = 0, 0, 0
        comp_time = 0
        total_queries = len(queries)

        # Loop over each query
        for idx, row in queries.iterrows():
            start_time = time.time()
            query = row["query"]
            doc_index = row["doc_index"]
            query = preprocess_query(query)

            # Rank documents for the current query
            ranked_indices, scores = rank_query_bm25(
                bm25_instance, query, doc_index, top_k=5, summarize_docs=False, only_scores=True
            )
            elapsed_time = time.time() - start_time
            comp_time += elapsed_time
            # Check if the doc_index is in the top 10, 5, 3, and 1
            if doc_index in ranked_indices:
                if doc_index in ranked_indices[:5]:
                    top5 += 1
                if doc_index in ranked_indices[:3]:
                    top3 += 1
                if doc_index in ranked_indices[:1]:
                    top1 += 1

        # Update timing and performance metrics
        results[i, :] += [top5 / total_queries, top3 / total_queries, top1 / total_queries]
        average_times[i] += comp_time / total_queries
        i += 1

# Calculate the average precision and speed for each p
results /= num_iterations  # Take the average over 20 iterations
average_times /= num_iterations  # Average the time taken
average_times_ms = [time * 1000 for time in average_times]  # Convert to milliseconds

# Write the average precision and average times to a file
with open("bm25_results.txt", "w") as f:
    for i, p in enumerate(p_values):
        f.write(f"Results for p={p}:\n")
        f.write(f"Average Precision at 5: {results[i, 0]:.4f}\n")
        f.write(f"Average Precision at 3: {results[i, 1]:.4f}\n")
        f.write(f"Average Precision at 1: {results[i, 2]:.4f}\n")
        f.write(f"Average Time taken: {average_times_ms[i]:.4f} ms\n\n")


In [66]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

# Create a PDF to save the plots
with PdfPages('precision_at_k.pdf') as pdf_pages:

    # First plot (Precision at different ranks)
    fig, ax = plt.subplots(figsize=(10, 6))
    p_values = [0, 0.1, 0.3, 0.5, 0.7, 0.9]
    labels = ['Precision at 5', 'Precision at 3', 'Precision at 1']

    for i in range(3):
        ax.plot(p_values, average_precision[:, i], label=labels[i], marker='o')

    ax.set_ylim(0, 1)
    ax.set_xlim(0, 1)
    ax.set_xlabel('p Value')
    ax.set_ylabel('Precision')
    ax.set_title('Precision at different ranks (1, 3, 5) for different p values')
    ax.legend()

    plt.grid(True)
    pdf_pages.savefig(fig)  # Save the first plot
    plt.close(fig)

In [68]:
    # Second plot (Computation Time for different p values)
with PdfPages('computation_time.pdf') as pdf_pages:
        fig, ax2 = plt.subplots(figsize=(10, 6))
    
        ax2.plot(p_values, average_times_ms, marker='o', color='b')
    
        ax2.set_xlabel('p Value')
        ax2.set_ylabel('Time (ms)')
        ax2.set_title('Computation Time for different p values')
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 20)  # Set y-axis to range from 0 to 50
        ax2.legend()
    
        plt.grid(True)
        pdf_pages.savefig(fig)  # Save the second plot
        plt.close(fig)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
print(average_times)

In [ ]:
count=0
for i in range(len(df)):
    article = df.iloc[i]["content"]
    if len(article) < 100:
        count+=1
        print(article)
print(count)

In [ ]:
df.iloc[33]["content"]

In [ ]:
df.iloc[528]["content"]

In [ ]:
df.iloc[55]["reduced_content_p=0.5"]

In [ ]:
queries.iloc[22]["query"]

In [ ]:
df.iloc[161]

In [ ]:
df.iloc[218]

In [ ]:
df.iloc[201]["content"]

In [ ]:
df.iloc[462]["content"]